In [1]:
from dotenv import load_dotenv
#load the .env : 
load_dotenv(r'C:\Users\Rahul\Documents\FREELANCING\1.AppliedSkil\Gen_AI_learning\.env')

True

## Embedding models

The Embeddings class is a class designed for interfacing with text embedding models. There are lots of embedding model providers (OpenAI, Cohere, Hugging Face, etc) - this class is designed to provide a standard interface for all of them.

Embeddings create a vector representation of a piece of text. This is useful because it means we can think about text in the vector space, and do things like semantic search where we look for pieces of text that are most similar in the vector space.

The base Embeddings class in LangChain provides two methods: one for embedding documents and one for embedding a query. The former takes as input multiple texts, while the latter takes a single text. The reason for having these as two separate methods is that some embedding providers have different embedding methods for documents (to be searched over) vs queries (the search query itself).

In [2]:
docs = [
    "cats eat and sleep",
    "dogs eat and bark",
    "cars drive fast",
    "vehicles include trucks and cars"
]

Embeddings create a vector representation of a piece of text. This is useful because it means we can think about text in the vector space, and do things like semantic search where we look for pieces of text that are most similar in the vector space.

The base Embeddings class in LangChain provides two methods: one for embedding documents and one for embedding a query. The former, `.embed_documents`, takes as input multiple texts, while the latter, `.embed_query`, takes a single text. The reason for having these as two separate methods is that some embedding providers have different embedding methods for documents (to be searched over) vs queries (the search query itself).

- `.embed_query`  will return a list of floats,
- `.embed_documents` returns a list of lists of floats.

### Open AI Embedding Models

LangChain enables us to access Open AI embedding models which include the newest models: a smaller and highly efficient `text-embedding-3-small` model, and a larger and more powerful `text-embedding-3-large` model.

In [3]:
from langchain_openai import OpenAIEmbeddings

# details here: https://openai.com/blog/new-embedding-models-and-api-updates
openai_embed_model = OpenAIEmbeddings(model='text-embedding-3-small')

In [4]:
embeddings = openai_embed_model.embed_documents(docs)

In [7]:
len(embeddings)

4

In [5]:
len(embeddings[0])

1536

In [6]:
print(embeddings[0])

[-0.014153297990560532, -0.004957524593919516, -0.009005471132695675, 0.011624541133642197, 0.020204251632094383, 0.02673257328569889, 0.023842565715312958, 0.03506714850664139, -0.005047837737947702, 0.0032399699557572603, 0.050188083201646805, 0.0012345430441200733, 0.031015977263450623, 0.04696263000369072, 0.0034576880279928446, 0.016759466379880905, 0.008082990534603596, 0.012179318815469742, -0.0069992379285395145, 0.03548000752925873, 0.037234656512737274, -0.028977490961551666, 0.03215133771300316, -0.035841260105371475, -0.029699992388486862, 0.054755330085754395, -0.008528104051947594, 0.03354473412036896, 0.012121261097490788, -0.029338741675019264, 0.024861808866262436, -0.040692344307899475, -0.011418111622333527, -0.04082136228680611, 0.011385857127606869, 0.02306845597922802, 0.02023005485534668, -0.03142883628606796, -0.006767005193978548, 0.01330177765339613, -0.009347369894385338, 0.014733879826962948, 0.021868586540222168, -0.00945703499019146, -0.009063529781997204,

In [7]:
docs

['cats eat and sleep',
 'dogs eat and bark',
 'cars drive fast',
 'vehicles include trucks and cars']

In [8]:
from sklearn.metrics.pairwise import cosine_similarity

sim_matrix = cosine_similarity(embeddings)
sim_matrix

array([[1.        , 0.61257608, 0.20355526, 0.19820971],
       [0.61257608, 1.        , 0.26200263, 0.20038633],
       [0.20355526, 0.26200263, 1.        , 0.39913189],
       [0.19820971, 0.20038633, 0.39913189, 1.        ]])

## Open Source Embedding Models on HuggingFace

`langchain-huggingface` integrates seamlessly with LangChain, providing an efficient and effective way to utilize Hugging Face models within the LangChain ecosystem.

`HuggingFaceEmbeddings`uses `sentence-transformers` embeddings. It computes the embedding locally, using your computer resources and allows you to access open or open source embedding LLMs hosted on HuggingFace.

In [3]:
from langchain_huggingface import HuggingFaceEmbeddings

# check out model details here: https://huggingface.co/mixedbread-ai/mxbai-embed-large-v1
model_name = "mixedbread-ai/mxbai-embed-large-v1"

hf_embeddings = HuggingFaceEmbeddings(
    model_name=model_name,
)

README.md:   0%|          | 0.00/114k [00:00<?, ?B/s]

In [6]:
embeddings = hf_embeddings.embed_documents(docs)

In [7]:
len(embeddings)

4

In [8]:
len(embeddings[0])

1024

In [16]:
docs

['cats eat and sleep',
 'dogs eat and bark',
 'cars drive fast',
 'vehicles include trucks and cars']

In [17]:
sim_matrix = cosine_similarity(embeddings)
sim_matrix

array([[1.        , 0.52516037, 0.34221861, 0.33691799],
       [0.52516037, 1.        , 0.31728606, 0.33082586],
       [0.34221861, 0.31728606, 1.        , 0.72253335],
       [0.33691799, 0.33082586, 0.72253335, 1.        ]])

## Build a small search engine!

### Load Knowledgebase documents

In [9]:
documents = [
    'Quantum mechanics describes the behavior of very small particles.',
    'Photosynthesis is the process by which green plants make food using sunlight.',
    "Shakespeare's plays are a testament to English literature.",
    'Artificial Intelligence aims to create machines that can think and learn.',
    'The pyramids of Egypt are historical monuments that have stood for thousands of years.',
    'Biology is the study of living organisms and their interactions with the environment.',
    'Music therapy can aid in the mental well-being of individuals.',
    'The Milky Way is just one of billions of galaxies in the universe.',
    'Economic theories help understand the distribution of resources in society.',
    'Yoga is an ancient practice that involves physical postures and meditation.'
]

In [19]:
len(documents)

10

### Get document embeddings

In [10]:
document_embeddings = openai_embed_model.embed_documents(documents)

### Let's try to find the most similar document for one query

In [1]:
new_text = 'What is ML?'
new_text

'What is ML?'

In [12]:
query_embedding = openai_embed_model.embed_query(new_text)

In [13]:
cosine_similarities = cosine_similarity([query_embedding], document_embeddings)
cosine_similarities

array([[ 0.10208743,  0.09262359, -0.00534998,  0.63140291,  0.02341739,
         0.09320838,  0.1076584 ,  0.07002011,  0.05820363,  0.06611405]])

In [14]:
import numpy as np

documents[np.argmax(cosine_similarities[0])]

'Artificial Intelligence aims to create machines that can think and learn.'

### Create Search Engine function

In [15]:
def semantic_search_engine(query, embedder_model):
  query_embedding = embedder_model.embed_query(query)
  cos_scores = cosine_similarity([query_embedding], document_embeddings)[0]
  top_result_id = np.argmax(cos_scores)
  return documents[top_result_id]

### Try out the function

In [16]:
new_sentence = 'Tell me about AI'
semantic_search_engine(new_sentence, openai_embed_model)

'Artificial Intelligence aims to create machines that can think and learn.'

In [17]:
new_sentence = 'Do you know about the pyramids?'
semantic_search_engine(new_sentence, openai_embed_model)

'The pyramids of Egypt are historical monuments that have stood for thousands of years.'

In [18]:
new_sentence = 'How do plants survive?'
semantic_search_engine(new_sentence, openai_embed_model)

'Photosynthesis is the process by which green plants make food using sunlight.'